In [25]:
%matplotlib notebook

import pandas as pd
from fastkml import  kml
from matplotlib import pyplot as plt
from os import path
import requests
import shutil
from pprint import pprint
import pickle

In [20]:
# The wind farm data (140256 timestamps x 164 wind farms)
training_data_filename = "ercot_wind_power.pkl"
# Download the data if it doesn't exist
# if not path.exists(training_data_filename):
#     TODO figure out how to correctly download this pickle file, because this code doesn't work
#     r = requests.get("http://storage.googleapis.com/gridmatic/roscoe/" + training_data_filename, stream=True)
#     with open(training_data_filename, 'wb') as fin:
#         shutil.copyfileobj(r.raw, fin)
#         fin.close()
#     pprint(r.raw)
#     pickle(r.raw, training_data_filename)
ercot_wind_power = pd.read_pickle(training_data_filename)

In [21]:
# get the locations for each of the wind farms
ercot_locations = {}
with open("rtmLmpPoints.kml", 'rt', encoding="utf-8") as ercot_location_file:
    doc = ercot_location_file.read()
    k = kml.KML()
    k.from_string(doc)
    features = list(k.features())
    for folder in features[0].features():
        for feat in folder.features():
            coords_plus_name = list(feat.geometry.coords[0])
            ercot_locations[feat.name] = coords_plus_name[:2]
    ercot_location_file.close()

# Add in coordinates of the locations not included or found in the provided KML file
ercot_locations['ASTRA_UNIT1'] = [-102.071477, 34.786654]
ercot_locations['KINGMTN_ALL'] = [-102.241490, 31.208996]

In [22]:
# Plot the last few days of wind farm output for 4 of the wind farms
# ercot_wind_power.iloc[-168*4:,:4].plot()
# plt.ylabel("Wind power production (MWh)")

In [26]:
# We are provided GPS coordinates of the wind farms, but the names provided are slightly different from the data
# We use a custom mapping to match the data with the coordinates

import csv

with open("data_name_to_coord_name.csv", mode='r') as infile:
    reader = csv.reader(infile)
    name_mapping = dict()
    name_mapping = {row[0]:row[1] for row in reader}
    infile.close()

data_name_to_coords = {key:ercot_locations[name_mapping[key]] for key in name_mapping}

pprint(data_name_to_coords)

with open('data_name_to_coords.pkl', 'wb') as handle:
    pickle.dump(data_name_to_coords, handle, protocol=pickle.HIGHEST_PROTOCOL)

{'ANACACHO_ANA': [-100.15802, 29.215214], 'ASTRA_UNIT1': [-102.071477, 34.786654], 'BCATWIND_WIND_1': [-98.570491, 33.509188], 'BLSUMMIT_BLSMT1_5': [-99.367467, 34.293473], 'BLSUMMIT_BLSMT1_6': [-99.367467, 34.293473], 'BORDAS2_JAVEL2_A': [-98.931877, 27.358092], 'BORDAS2_JAVEL2_B': [-98.931877, 27.358092], 'BORDAS2_JAVEL2_C': [-98.931877, 27.358092], 'BORDAS_JAVEL18': [-98.931877, 27.358092], 'BORDAS_JAVEL20': [-98.931877, 27.358092], 'BRAZ_WND_WND1': [-101.146911, 32.949884], 'BRAZ_WND_WND2': [-101.146911, 32.949884], 'BRISCOE_WIND': [-101.374278, 34.371453], 'BRTSW_BCW1': [-98.363595, 33.073455], 'BUCKTHRN_UNIT1': [-98.353682, 32.376956], 'BUCKTHRN_UNIT2': [-98.353682, 32.376956], 'BUFF_GAP_UNIT1': [-100.115386, 32.298543], 'BUFF_GAP_UNIT2_1': [-100.115386, 32.298543], 'BUFF_GAP_UNIT2_2': [-100.115386, 32.298543], 'BUFF_GAP_UNIT3': [-100.115386, 32.298543], 'BULLCRK_WND1': [-101.605095, 32.906769], 'BULLCRK_WND2': [-101.605095, 32.906769], 'CALLAHAN_WND1': [-99.995602, 32.296385], '